# BioKotlin GFF Parsing using the FeatureTree Package
While the fundamental model of this package is substantially complete, many biological convenience functions using this model need to be implemented.

## Simple Parsing and Basic Queries

First, generate a `Genome` from a GFF3 file:

In [1]:
@file:DependsOn("../build/libs/BioKotlin-0.08-all.jar")
import biokotlin.featureTree.*
val genome = Genome.fromFile("resources/b73_shortened.gff")

Let's query a gene within this genome! The `byID` function is constant-time and returns a `Feature?`

In [2]:
val gene = genome.byID("Zm00001eb000010")!!

`gene` is a `Feature` instance because it contains the data in the 9 columns of a GFF file. Let's query some of this data.

In [3]:
println(""" 
Source: ${gene.source}
Start: ${gene.start}
biotype Attribute: ${gene.attribute("biotype")}
""".trimIndent())

Source: NAM
Start: 34617
biotype Attribute: [protein_coding]


`gene` is a `Parent` because it is also the root of a tree of `Feature`. Let's get access to the direct children of `gene`. In this case, there is only one child, a transcript.

In [4]:
val geneChildren = gene.children
val transcript = geneChildren.first()
println(transcript.id)

Zm00001eb000010_T001


Of course, you can also walk back up the tree with `parent`.

In [5]:
println("${transcript.parent == gene}")

true


It is often useful to apply an operation for every node below the `parent` on the tree, not only its immediate children. This is where `descendants` comes in, which produces a sequence of all nodes below the parent. These nodes are in depth-first, left-to-right order. This sequence can be combined with the powerful Kotlin collections framework to make a range of interesting queries.

In [6]:
gene.descendants().map { it.type }.toList()

[mRNA, five_prime_UTR, exon, exon, exon, exon, exon, exon, exon, exon, exon, CDS, three_prime_UTR, three_prime_UTR, three_prime_UTR]

`subtree` is quite similar except that it inclues the receiver as well. Observe the inclusion of "gene" in the output.

In [7]:
gene.subtree().map { it.type }.toList()

[gene, mRNA, five_prime_UTR, exon, exon, exon, exon, exon, exon, exon, exon, exon, CDS, three_prime_UTR, three_prime_UTR, three_prime_UTR]